<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/SQL/Analitics/part_01/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
!pip install calamine -q

ERROR: Could not find a version that satisfies the requirement calamine (from versions: none)
ERROR: No matching distribution found for calamine


In [6]:
file = "https://github.com/aleks-haksly/Simulative/blob/2838825651c222a0b6d05ffdaf6888645353b8f7/SQL/Analitics/part_01/product_metrics_retention.xlsx"

In [14]:
adv_company = pd.read_excel(file, sheet_name='input_data')

ValueError: Excel file format cannot be determined, you must specify an engine manually.